In [ ]:
from __future__ import annotations

import copy
from typing import Dict, List, Optional, Set, Tuple

import cvxpy as cp  # note: also need cvxopt installed
import numpy as np
import pandas as pd

# Setup

In [ ]:
# Define players and their point projections
PLAYERS = pd.DataFrame(
    [
        ["QB1", "QB", 20, 15, 4],
        ["QB2", "QB", 17, 16, 5],
        ["QB3", "QB", 16, 17, 6],
        ["RB1", "RB", 16, 14, 1],
        ["RB2", "RB", 12, 16, 2],
        ["RB3", "RB", 13, 14, 3],
    ],
    columns=["uid", "position", "week1", "week2", "adp"],
)
ALL_IDX = set(PLAYERS.index)
WEEK_COLS = [c for c in PLAYERS.columns if c.startswith("week")]
ADP_COL = "adp"

PLAYERS

In [ ]:
# Define draft order
DRAFT = [0, 1, 2, 2, 1, 0]
NUM_TEAMS = len(set(DRAFT))
NUM_PICKS = len(DRAFT)

In [ ]:
# Define constraints
NUM_PLAYERS_CONST = 2
POS_CONST = {"QB": 1, "RB": 1}

In [ ]:
def display_draft(picks_idx):
    for team, idx in picks_idx.items():
        print(f"--- Team {team} ---")
        roster = PLAYERS.loc[list(idx), ["uid"] + WEEK_COLS].set_index("uid")
        sum_points = roster.sum(axis=0).to_frame().rename({0: "sum_points"}, axis=1).T
        sum_points["min"] = sum_points.min(axis=1)
        display(roster)
        display(sum_points.sort_index(axis=1))

# Optimized Decision Tree

In [ ]:
def poss_opt_picks(team, picks_idx, picked_idx):
    # Get available players and those already picked by the team
    available_idx = ALL_IDX - picked_idx
    prev_picks_idx = picks_idx[team]
    available_idx |= prev_picks_idx
    available_idx = sorted(available_idx)

    # Get data
    uid_vals = PLAYERS.loc[available_idx, "uid"].values
    pos_vals = PLAYERS.loc[available_idx, "position"].values
    points_vals = PLAYERS.loc[available_idx, WEEK_COLS].values

    # The variable we are solving for. We define our output variable as a bool
    # since we have to make a binary decision on each player (pick or don't pick)
    roster = cp.Variable(len(available_idx), boolean=True)

    # Save constraints
    constraints = []

    # Our roster must be composed of exactly `num_players` players
    constraints.append(cp.sum(roster) == NUM_PLAYERS_CONST)

    # Define position constraints
    for pos, num in POS_CONST.items():
        is_pos = pos_vals == pos
        constraints.append(cp.sum(is_pos @ roster) == num)

    # Define constraints corresponding already picked players
    for idx in prev_picks_idx:
        did_pick = uid_vals == PLAYERS.loc[idx, "uid"]
        constraints.append(cp.sum(did_pick @ roster) == 1)

    # Define the objective
    weekly_points = roster @ points_vals
    min_weekly_points = cp.min(weekly_points)
    objective = cp.Maximize(min_weekly_points)

    # Solve
    problem = cp.Problem(objective, constraints)
    problem.solve(max_iters=25)

    # Get result
    roster_idx = roster.value
    if roster_idx is not None:
        opt_roster = np.array(available_idx)[roster_idx.astype(bool)]  # re-align indices
        result = set(opt_roster)
    else:
        result = set()

    return result

In [ ]:
class Node:
    def __init__(self, pick_num: int, picks_idx: Dict[int, Set[int]], parent: Optional[Node] = None):
        if pick_num < NUM_PICKS:
            self.team = DRAFT[pick_num]
        else:
            self.team = None
        self.pick_num = pick_num
        self.picks_idx = picks_idx
        self.picked_idx = set(sum([list(v) for v in picks_idx.values()], []))
        self.parent = parent
        self.children = []

    def _poss_picks_helper(self, poss_picks: Set[int], n: int = 5) -> Set[int]:
        # Remove current players
        curr_roster = set(self.picks_idx[self.team])
        poss_picks = poss_picks - curr_roster

        # Limit to top N by ADP; helps prevent tree from becoming unwieldy
        top = PLAYERS.loc[list(poss_picks), ADP_COL].sort_values()
        poss_picks = set(top.iloc[0:n].index)

        return poss_picks

    def branch(self, max_branch_size: int = 5) -> List[Node]:
        # Get info
        team, pick_num, picks_idx, picked_idx = self.team, self.pick_num, self.picks_idx, self.picked_idx

        # Optimize if possible
        children = []
        if team is not None:
            # Optimize as if we get all picks in a row
            poss_picks = poss_opt_picks(team, picks_idx, picked_idx)  # ~0.01 seconds/call for the simple QB/RB case
            poss_picks = self._poss_picks_helper(poss_picks, max_branch_size)

            # Optimize as if each original optimal player is already taken
            for p in poss_picks.copy():
                new_poss_picks = poss_opt_picks(team, picks_idx, picked_idx | {p})
                poss_picks |= new_poss_picks
            poss_picks = self._poss_picks_helper(poss_picks, max_branch_size)

            # Make children
            for p in poss_picks:
                temp_picks_idx = copy.deepcopy(picks_idx)
                temp_picks_idx[team] |= {p}
                node = Node(pick_num + 1, temp_picks_idx, self)
                children.append(node)

        # Save
        self.children = children

        return children

    def __repr__(self) -> str:
        return f"Node(team={self.team}, pick_num={self.pick_num})"

In [ ]:
def traverse_node(root: Node, max_depth: int = len(DRAFT), max_branch_size: int = 5) -> List[Node]:
    # Traverse
    children = [root]
    depth = -1
    while len(children) > 0 and depth < max_depth:
        temp = []
        for child in children:
            temp += child.branch(max_branch_size)
        last_children = children
        children = temp
        depth += 1

    return last_children


def score_picks(idx: Set[int]) -> float:
    roster = PLAYERS.loc[list(idx), WEEK_COLS]
    return roster.sum(axis=0).min()


def opt_roster(
    pick_num: int, picks_idx: Dict[int, Set[int]], max_depth: int = len(DRAFT), max_branch_size: int = 5
) -> Tuple[float, Set[int], picks_idx : Dict[int, Set[int]]]:
    # Fully traverse root
    root = Node(pick_num, picks_idx)
    children = traverse_node(root, max_depth, max_branch_size)

    # Traverse up the tree, making optimal decisions for each team along the way
    # Note: this isn't terribly slow (at least not compared to the solver in `traverse_node`)
    num_explored = 0
    prev_level = set([c.parent for c in children])
    while len(prev_level) > 1:  # terminate at the root
        next_prev_level = set()
        for n in prev_level.copy():  # prevent mutation during loop
            # Find optimal child for the node's team
            scores = []
            for c in n.children:
                num_explored += 1
                scores.append(score_picks(c.picks_idx[n.team]))
            best = n.children[np.argmax(scores)]

            # Promote the best child
            parent_idx = n.parent.children.index(n)
            n.parent.children[parent_idx] = best

            # Append to the next iteration of previous levels
            next_prev_level |= {n.parent}

        # Prepare for next iteration
        prev_level = next_prev_level

    # Find the final optimum
    root = list(prev_level)[0]
    scores = []
    for c in root.children:
        num_explored += 1
        scores.append(score_picks(c.picks_idx[root.team]))
    max_min_idx = np.argmax(scores)
    best = root.children[max_min_idx]

    # Output optimal roster
    max_min_points = scores[max_min_idx]
    all_opt_picks = best.picks_idx
    opt_picks = all_opt_picks[root.team]

    # Log
    print(f"Explored {num_explored} nodes to find optimal picks.")

    return max_min_points, opt_picks, all_opt_picks

In [ ]:
# Traverse one pick
# picks_idx = {k: set() for k in range(NUM_TEAMS)}
# children = traverse_node(Node(0, picks_idx), max_depth=1)
# for c in children:
#     print(c.picks_idx)

In [ ]:
%%time

# Specify optimizer parameters
max_depth = len(DRAFT)  # how deep to optimize; deeper is more computationally intensive (i.e. more nodes)
max_branch_size = 5  # limit branch size at each node; larger is more computationally intensive (i.e. more nodes)

# Optimize entire draft
pick_num = 0  # first overall pick
picks_idx = {k: set() for k in range(NUM_TEAMS)}
max_min_points, opt_picks, all_opt_picks = opt_roster(
    pick_num, picks_idx, max_depth=max_depth, max_branch_size=max_branch_size
)

display_draft(all_opt_picks)

In [ ]:
%%time

# Specify optimizer parameters
max_depth = len(DRAFT)  # how deep to optimize; deeper is more computationally intensive (i.e. more nodes)
max_branch_size = 5  # limit branch size at each node; larger is more computationally intensive (i.e. more nodes)

# Optimize after team 0 took RB3 with the first pick
pick_num = 1  # second overall pick
picks_idx = {k: set() for k in range(NUM_TEAMS)}
picks_idx[0] |= {PLAYERS[PLAYERS["uid"] == "RB3"].index[0]}
max_min_points, opt_picks, all_opt_picks = opt_roster(
    pick_num, picks_idx, max_depth=max_depth, max_branch_size=max_branch_size
)

display_draft(all_opt_picks)